# Imports

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
import datedelta
from tqdm.notebook import tqdm
from pymystem3 import Mystem
import nltk
from nltk import word_tokenize, ngrams
from nltk.corpus import stopwords
from collections import Counter
import re
import pickle
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import guidedlda
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import KFold
from scipy import stats
import xgboost
from sklearn.feature_extraction.text import TfidfVectorizer

# Guided LDA

Здесь я использую заранее отобранный сэмпл (сбалансированный по СМИ) и привожу его к виду для LDA

In [ ]:
text_tokens = pickle.load(open('model/text_tokens_100k.pkl', 'rb'))
# подгружаю сформированный ранее словарь стоп-слов ()
stop_words = pd.read_json('stop_words_russian.json', encoding='utf-8')[0].tolist()
stop_words = ' | '.join(stop_words)
trash_phrases = pickle.load(open('model/trash_phrases.pkl', 'rb'))

In [ ]:
stem = Mystem()
news_lem, acc_all = [], []
#Обрабатываю по 1к, т.к. по одному слишком медленно https://habr.com/ru/post/503420/
for i in tqdm(range(0, len(text_tokens), 1000)):
    news_batch = news[i:i+1000]
    text = ' div '.join(news_batch) # объединяю в одну строку
    text = stem.lemmatize(text) # лемматизирую
    text = ''.join(text) # объединяю токены назад (т.к. нужно разделить по 'div')
    text = re.sub(stop_words, " ", text) # убираю стоп-слова
    text = re.sub(trash_phrases, " ", text) # убираю мусор
    text = re.sub('[^A-Za-zА-Яа-я\s]', '', text) # убираю все символы, кроме букв, цифр, знаков препинания и пробелов
    text = text.split('div') # разделяю обратно по div
    news_lem.extend(text)
# привожу к виду matrix of token counts для последующего использования в алгоритме (для LDA tf-idf и иные не подходят, 
# более сложные занимают больше времени, в условиях ограниченнного времени это оптимальный выбор)
X = vectorizer.fit_transform(news_lem).astype(np.int)
pickle.dump(vectorizer, open('model/vectorizer.pkl', 'wb'))

Так как изначально буду использовать semi-supervised Guided (labeled) LDA, то необходимо подгрузить заранее сформированный мною словарь экономических слов (был сделан на основе словаря экономических новостей базы, экспертно взяты из него наиболее частые экономические термы)

In [ ]:
ecwords = pd.read_excel('econom_dict.xlsx')['1gram'].tolist()
# убираем из слова экономических термов те, которые не встретились в основном словаре
ecwords = [x for x in ecwords if x in list(word2id.keys())]

seed_topics = {}
for word in ecwords:
    seed_topics[word2id[word]] = 0

Строю Guided LDA модели от 2 до 14 тем, когерентность не смотрю, т.к. есть размеченный датасет для supervised learning, на котором я и проверю

In [ ]:
for i in tqdm(range(2, 15)):
    model = guidedlda.GuidedLDA(n_topics=i, n_iter=300, random_state=7, refresh=20,alpha=0.01,eta=0.01)
    model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)
    pickle.dump(model, open(f'model/model_{i}.pkl', 'wb'))

# Supervised learning

Заранее была проведена работа по разметке данных по ключевым словам. Были выделены 6 тем:  
+ 0 - экономика  
+ 1 - политика, в мире  
+ 2 - общество  
+ 3 - культура  
+ 4 - проишествия  
+ 5 - спорт  

Датасет формировался следующим образом: 20к по каждой из тем. Далее был использован tf-idf для feature extraction, минимальное количество встречи слова - 100, максимальная частота - 0,8 (встречается менее, чем в 80% документов)

X был обработан аналогично, как для Guided LDA (стеммизирован+очищен от мусора/лишних знаков и т.д.)

y - список меток классов

In [ ]:
tfidf = pickle.load(open('model_supervised/tfidf.pkl', 'rb'))
dataset = pickle.load(open('markups keys/data/dataset.pkl', 'rb'))
X = tfidf.transform(dataset)
y = pickle.load(open('markups keys/data/y.pkl', 'rb'))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Далее идем по списку supervised моделей и сравниваем по f1-srore, precision, recall. Плюсом проводим guided LDA через валидационную часть для получения сравнимых прогнозных мощностей моделей 

Каждую модель провожу через cv, 5 folds с перемешиванием. CV по метрике f1-macro, т.к. выборка сбалансированная, то можно использовать данную метрику

In [ ]:
kfold_5 = KFold(n_splits=5, shuffle=True)

## Softmax

поиск по сетке идет на основе используемого регуляризатора (l1, l2 или без), константы, а также

In [ ]:
grid = {'penalty': ['l1', 'l2', None],
        "C":np.logspace(-4,4,20),
        'fit_intercept': [True, False]}

lr = LogisticRegression(n_jobs=4)
clf_lr = RandomizedSearchCV(lr, 
                            param_distributions=grid,
                            cv=kfold_5,  
                            n_iter=5,
                            scoring='f1_macro', 
                            error_score=0, 
                            verbose=3, 
                            n_jobs=-1)
clf_lr.fit(X_train, y_train)

In [ ]:
y_pred_lr = clf_lr.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_lr.tolist()))

## SVM

In [ ]:
from sklearn.svm import SVC

grid = {'kernel': ['linear'],
        'gamma': [1e-3, 1e-4],
        'C': [1, 100, 1000]}

model_svm = SVC(random_state=42)
clf_svm = RandomizedSearchCV(model_svm, 
                             param_distributions = grid,
                             cv=kfold_5,  
                             n_iter=5,
                             scoring='f1_macro', 
                             error_score=0, 
                             verbose=3, 
                             n_jobs=-1)
clf_svm.fit(X_train, y_train)

In [ ]:
y_pred_svm = clf_svm.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_svm.tolist()))

## Random forest

In [ ]:
param_dist = {'n_estimators': stats.randint(50, 2000),
              'max_depth': stats.randint(10, 110),
              'max_features': ['auto', 'sqrt'],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4],
              'bootstrap': [True, False]
             }
numFolds = 5
kfold_5 = KFold(n_splits = numFolds, shuffle = True)

clf = RandomizedSearchCV(rf, 
                         param_distributions = param_dist,
                         cv = kfold_5,  
                         n_iter = 5,
                         scoring = 'f1_macro', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred.tolist()))

## xgboost

In [ ]:
model = xgboost.XGBClassifier(nthread=4, objective='binary:logistic')
param_dist = {'n_estimators': stats.randint(20, 500),
              "colsample_bytree": [0.6, 0.8, 1.0],
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.8),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4],
              "gamma": [0, 0.1, 0.3,0.4],
             }
numFolds = 5
kfold_5 = KFold(n_splits = numFolds, shuffle = True)

clf = RandomizedSearchCV(model, 
                         param_distributions = param_dist,
                         cv = kfold_5,  
                         n_iter = 5,
                         scoring = 'f1_macro', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred.tolist()))

## xgboost 2-class

Также для сравнения эффективности использования модель с большим количеством классов для лучшей разделимости тем была обучена 2-классовая модель xgboost

In [ ]:
y_train_ec = [0 if e==0 else 1 for e in y_train]
y_test_ec = [0 if e==0 else 1 for e in y_test]

In [ ]:
model_ec = xgboost.XGBClassifier(nthread=4, objective='binary:logistic')
param_dist = {'n_estimators': stats.randint(20, 500),
              "colsample_bytree": [0.6, 0.8, 1.0],
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.8),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4],
              "gamma": [0, 0.1, 0.3,0.4],
             }
numFolds = 5
kfold_5 = KFold(n_splits = numFolds, shuffle = True)

clf_ec = RandomizedSearchCV(model_ec, 
                         param_distributions = param_dist,
                         cv = kfold_5,  
                         n_iter = 5,
                         scoring = 'f1_macro', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

clf_ec.fit(X_train, y_train_ec)

In [ ]:
y_pred_ec = clf_ec.predict(X_test)
print(classification_report(y_test_ec, y_pred_ec.tolist()))